In [1]:
import torch.nn as nn
import torch
from PIL import Image
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os

In [2]:
# Step 1
# Get the image and change every image to an 256-dimention vector
dataSet = np.zeros([10, 256])
for i in range(0, 10):
    inputImageDir = './input/' + str(i) + '.png'
    inputImage = Image.open(inputImageDir)
    inputImage = inputImage.convert("1")
    inputImage.save(inputImageDir)
    data = inputImage.getdata()
    array = np.array(data)/255
    dataSet[i] = array
dataSet = np.array(dataSet)

In [3]:
# Step 2
# define a neural network
class Perceptron(nn.Module):

    def __init__(self, input_size, num_classes):
        super(Perceptron, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        self.activate = nn.Sigmoid()

    def forward(self, x):
        res = self.linear(x)
        res = self.activate(res)
        return res

In [4]:
class DigitDataset(Dataset):
    def __init__(self, dataset, label_list):
        self.dataset = dataset
        self.label_list = label_list
    def __len__(self):
        return len(self.label_list)
    def __getitem__(self, idx):
        data = self.dataset[idx]
        label = self.label_list[idx]
        return {
            'data': torch.from_numpy(data).float(),
            'label': torch.from_numpy(label).float()
        }

In [5]:
# Parameters of training
input_size = 256
num_classes = 256
learning_rate = 0.001
batch_size = 10
num_epochs = 600

# Load the dataset
train_dataset = DigitDataset(dataset = dataSet, label_list = dataSet)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
device = torch.device('cpu')
model = Perceptron(input_size=input_size, num_classes=num_classes).to(device)

if not os.path.exists('./models'):
    os.mkdir('./models')
torch.save(model, './models/net_untrained.pkl')

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [6]:
# Step 3
def train(dataloader, model, num_epochs):
    for epoch in range(num_epochs):
        losses = []
        ERROR_Train = [] 
        model.train() 
        for i, data in enumerate(dataloader, 0):
            model.zero_grad()
            real_cpu, label_cpu = data['data'], data['label']
#             if torch.cuda.is_available():
#                 real_cpu = real_cpu.cuda() 
#                 label_cpu = label_cpu.cuda()
            real = real_cpu
            label = label_cpu
            inputv = Variable(real)
            labelv = Variable(label)
            output = model(inputv)
            err = criterion(output, labelv) 
            err.backward() 
            optimizer.step() 

            losses.append(err.data.item())
            error = label - output.data
#             print(error.shape)
            ERROR_Train.extend(error)
#         print(ERROR_Train)
        MAE = torch.mean(torch.abs(torch.stack(ERROR_Train)))
        ME = torch.mean(torch.stack(ERROR_Train))
        STD = torch.std(torch.stack(ERROR_Train)) 
        if epoch % 10 == 0 or epoch == num_epochs - 1:
            print('[%d/%d] Loss: %.4f MAE: %.4f Mean Error: %.4f STD: %.4f' % (epoch, num_epochs, np.average(losses), MAE, ME, STD))
    return output, model

# Start training        
output, model = train(train_loader, model, num_epochs)
# print(output.type)

[0/600] Loss: 0.2661 MAE: 0.4995 Mean Error: 0.4124 STD: 0.3100
[10/600] Loss: 0.0441 MAE: 0.1588 Mean Error: 0.0965 STD: 0.1866
[20/600] Loss: 0.0308 MAE: 0.0915 Mean Error: 0.0354 STD: 0.1720
[30/600] Loss: 0.0282 MAE: 0.0782 Mean Error: 0.0211 STD: 0.1666
[40/600] Loss: 0.0265 MAE: 0.0734 Mean Error: 0.0170 STD: 0.1618
[50/600] Loss: 0.0249 MAE: 0.0707 Mean Error: 0.0144 STD: 0.1573
[60/600] Loss: 0.0236 MAE: 0.0689 Mean Error: 0.0136 STD: 0.1529
[70/600] Loss: 0.0223 MAE: 0.0673 Mean Error: 0.0131 STD: 0.1486
[80/600] Loss: 0.0210 MAE: 0.0656 Mean Error: 0.0128 STD: 0.1445
[90/600] Loss: 0.0199 MAE: 0.0640 Mean Error: 0.0124 STD: 0.1405
[100/600] Loss: 0.0188 MAE: 0.0624 Mean Error: 0.0120 STD: 0.1367
[110/600] Loss: 0.0178 MAE: 0.0609 Mean Error: 0.0118 STD: 0.1329
[120/600] Loss: 0.0169 MAE: 0.0593 Mean Error: 0.0116 STD: 0.1293
[130/600] Loss: 0.0160 MAE: 0.0577 Mean Error: 0.0113 STD: 0.1259
[140/600] Loss: 0.0151 MAE: 0.0563 Mean Error: 0.0110 STD: 0.1225
[150/600] Loss: 0.014

In [7]:
# Step 4
# Step 4a
# Export the image after training
# Before executing this block, create a folder called "output"
if not os.path.exists('./output'):
    os.mkdir('./output')
output_np = output.detach().numpy()
print(output_np.shape)
torch.save(model, './models/net_trained.pkl')
output_dataset = np.zeros([10, 256])
for i in range(10):
    output_img = output_np[i].reshape(16, 16)*255
    img = Image.fromarray(np.uint8(output_img))
    img = img.convert("1")
    output_path = './output/' + str(i) + '.png'
    img.save(output_path)
    data = img.getdata()
    array = np.array(data)/255
    output_dataset[i] = array

(10, 256)


In [8]:
# Step 4b
# Calculate Fh
def calculateFh(input_dataset, output_dataset):
    x, y = input_dataset.shape
    Fh_denominator = 0    # Fh分母
    Fh_numerator = 0      # Fh分子
    Fh_array = np.zeros([x])
    for j in range(x):
        for i in range(y):
            if input_dataset[j][i] == 0:
                Fh_denominator = Fh_denominator + 1
                if output_dataset[j][i] == 0:
                    Fh_numerator = Fh_numerator + 1
        Fh = Fh_numerator / Fh_denominator
        Fh_array[j] = Fh
    return Fh_array

In [9]:
# Calculate Ffa
def calculateFfa(input_dataset, output_dataset):
    x, y = input_dataset.shape
    Ffa_denominator = 0    # Ffa分母
    Ffa_numerator = 0      # Ffa分子
    Ffa_array = np.zeros([x])
    for j in range(x):
        for i in range(y):
            if input_dataset[j][i] == 1:
                Ffa_denominator = Ffa_denominator + 1
            if output_dataset[j][i] == 0 and input_dataset[j][i] == 1:
                Ffa_numerator = Ffa_numerator + 1
        Ffa = Ffa_numerator / Ffa_denominator
        Ffa_array[j] = Ffa
    return Ffa_array

In [10]:
Fh_array = calculateFh(dataSet, output_dataset)
Ffa_array = calculateFfa(dataSet, output_dataset)
print(Fh_array)
print(Ffa_array)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.004329   0.00210526 0.00141243 0.0021254  0.00170213 0.00142146
 0.001221   0.0010644  0.00094877 0.00085507]


In [11]:
# TODO: Step 4c: Graph Fh as a function of Ffa for each exemplar in the input dataset

In [12]:
# Step 5
def gaussian_noise(img, mean, sigma):
    # Generate gauss noise
    noise = np.random.normal(mean, sigma, img.shape)
    # Add the noise to image
    gaussian_out = img + noise
    # Make the value between 0 and 1
    gaussian_out = np.clip(gaussian_out, 0, 1)
    return gaussian_out

gaussian_dataset = np.zeros([9, 10, 256])
std = [0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1]
if not os.path.exists('./input_noise/'):
    os.mkdir('./input_noise/')
for j in range(9):
    if not os.path.exists('./input_noise/' + str(std[j])):
        os.mkdir('./input_noise/' + str(std[j]))
    for i in range(10):
        inputImage = dataSet[i]
        gaussian_data = gaussian_noise(inputImage, 0, std[j])
        img = gaussian_data.reshape(16, 16)*255
        img = Image.fromarray(np.uint8(img))
        img.convert("1")
        inputImageDir = './input_noise/' + str(std[j]) + '/' + str(i) + '.png'
        img.save(inputImageDir)
        gaussian_dataset[j][i] = gaussian_data
gaussian_dataset = np.array(gaussian_dataset)


Fh_noise_array = np.zeros([9, 10])
Ffa_noise_array = np.zeros([9, 10])

# Train 9 datasets with noise
if not os.path.exists('./output_noise/'):
    os.mkdir('./output_noise/')
for j in range(9):
    train_noise_dataset = DigitDataset(dataset = gaussian_dataset[j], label_list = dataSet)
    train_noise_loader = DataLoader(dataset=train_noise_dataset, batch_size=batch_size, shuffle=False)
    print('Training dataset with noise standard deviation ' + str(std[j]))
#     model_noise = torch.load('./models/net_untrained.pkl') #  Load the model that trained before
    model_noise = Perceptron(input_size=input_size, num_classes=num_classes).to(device)
    output_noise, model_noise = train(train_noise_loader, model_noise, num_epochs)   # Train
    torch.save(model_noise, './models/net_trained_' + str(std[j]) + '.pkl')
    model_noise = torch.load('./models/net_trained_' + str(std[j]) + '.pkl')
#     output_noise = model_noise(torch.from_numpy(gaussian_dataset[j]).float()) # Use the model trained before to test
    print('------------------------------------')
    output_noise = model_noise(torch.from_numpy(dataSet).float())
    output_noise_np = output_noise.detach().numpy()     # Get the output
#     print(output_noise_np)
    output_noise_dataset = np.zeros([10, 256])
#     Make the output only has 0 or 1
    
    if not os.path.exists('./output_noise/' + str(std[j])):
        os.mkdir('./output_noise/' + str(std[j]))
    
    
    
    for i in range(10):
        output_noise_img = output_noise_np[i].reshape(16, 16)*255
        img = Image.fromarray(np.uint8(output_noise_img))
        img = img.convert("1")
        output_path = './output_noise/' + str(std[j]) + '/' + str(i) + '.png'
        img.save(output_path)
        data = img.getdata()
        array = np.array(data)/255
        output_noise_dataset[i] = array
#     Calculate Fh and Ffa
    Fh = calculateFh(dataSet, output_noise_dataset)
    Ffa = calculateFfa(dataSet, output_noise_dataset)
    Fh_noise_array[j] = Fh
    Ffa_noise_array[j] = Ffa
print('------------Fh_noise_array------------')
print(Fh_noise_array)
print('------------Ffa_noise_array------------')
print(Ffa_noise_array)

Training dataset with noise standard deviation 0.001
[0/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[10/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[20/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[30/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[40/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[50/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[60/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[70/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[80/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[90/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[100/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[110/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[120/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[130/600] Loss: 0.2683 MAE: 0.5007 Mean Error: 0.4078 STD: 0.3195
[140/600] Loss: 0.2683 MAE: 0.5007

Training dataset with noise standard deviation 0.003
[0/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[10/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[20/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[30/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[40/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[50/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[60/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[70/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[80/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[90/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[100/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[110/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[120/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[130/600] Loss: 0.2704 MAE: 0.5026 Mean Error: 0.4159 STD: 0.3122
[140/600] Loss: 0.2704 MAE: 0.5026

[30/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[40/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[50/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[60/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[70/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[80/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[90/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[100/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[110/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[120/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[130/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[140/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[150/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[160/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[170/600] Loss: 0.2597 MAE: 0.4950 Mean Error: 0.4110 STD: 0.3014
[180/600] Loss: 0

[110/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[120/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[130/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[140/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[150/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[160/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[170/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[180/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[190/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[200/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[210/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[220/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[230/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[240/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[250/600] Loss: 0.2660 MAE: 0.4966 Mean Error: 0.4119 STD: 0.3105
[260/600] 

[170/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[180/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[190/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[200/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[210/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[220/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[230/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[240/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[250/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[260/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[270/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[280/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[290/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[300/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[310/600] Loss: 0.2702 MAE: 0.5057 Mean Error: 0.4193 STD: 0.3073
[320/600] 

In [13]:
# TODO: Step 6: Display Data from your Tests in Step 5